<a href="https://colab.research.google.com/github/GiovaniValdrighi/school_eval_ethical/blob/main/notebooks/data_cleaning_for_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [67]:
def bar_plot_counts(df, col):
  fig = plt.figure(figsize = (8, 3))
  unique_values = df[col].unique().tolist()
  counts = [np.sum(df[col] == u) for u in unique_values]
  plt.bar(list(range(len(counts))), counts)
  plt.xticks(list(range(len(counts))), labels = unique_values)
  plt.ylabel("count")
  plt.title(f"Distribution of values for {col} column")
  plt.show()

def hist_plot(df, col):
  fig = plt.figure(figsize = (8, 3))
  plt.hist(df[col].values, bins = 100)
  plt.ylabel("count")
  plt.title(f"Distribution of values for {col} column")
  plt.show()

In [72]:
df = pd.read_csv("/content/drive/MyDrive/courses/ai_ethics/school_eval/data/microdados_saeb_2019/clean_schools_v2.csv")
df.head()

,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_FINAL,NIVEL_SOCIO_ECONOMICO,NU_MATRICULADOS_CENSO_9EF,NU_PRESENTES_9EF,TAXA_PARTICIPACAO_9EF,...,TX_RESP_Q245,TX_RESP_Q246,TX_RESP_Q247,TX_RESP_Q248,TX_RESP_Q249,TX_RESP_Q250,TX_RESP_Q251,TX_RESP_Q252,TX_RESP_Q253,MEDIA
0,1.0,11.0,6311030.0,2.0,1.0,48.9,Nível IV,14.0,14.0,1.000000,...,invalido,invalido,invalido,invalido,invalido,invalido,invalido,C,C,242.350
1,1.0,11.0,6311030.0,2.0,1.0,55.6,Nível III,16.0,14.0,0.875000,...,invalido,invalido,invalido,invalido,invalido,invalido,invalido,D,D,270.780
2,1.0,11.0,6311030.0,2.0,2.0,45.5,Nível IV,12.0,10.0,0.833333,...,B,B,B,B,B,A,B,D,D,245.215
3,1.0,11.0,6311030.0,2.0,1.0,72.9,Nível IV,209.0,195.0,0.933014,...,invalido,invalido,invalido,invalido,invalido,invalido,invalido,D,E,270.980
4,1.0,11.0,6311030.0,2.0,1.0,36.5,Nível IV,12.0,10.0,0.833333,...,invalido,invalido,invalido,invalido,invalido,invalido,invalido,D,D,240.935


## Converting columns values

In [73]:
id_regiao = {1 : "norte", 2: "nordeste", 3:"sudeste", 4:"sul", 5:"centro-oeste"}
id_uf = {11: 'RO',
 12: 'AC',
 13: 'AM',
 14: 'RR',
 15: 'PA',
 16: 'AP',
 17: 'TO',
 21: 'MA',
 22: 'PI',
 23: 'CE',
 24: 'RN',
 25: 'PB',
 26: 'PE',
 27: 'AL',
 28: 'SE',
 29: 'BA',
 31: 'MG',
 32: 'ES',
 33: 'RJ',
 35: 'SP',
 41: 'PR',
 42: 'SC',
 43: 'RS',
 50: 'MS',
 51: 'MT',
 52: 'GO',
 53: 'DF'}
id_area = {1: "capital", 2:"interior"}
id_localizacao = {1 : "urbana", 2:"rural"}
df["ID_MUNICIPIO"] = df["ID_MUNICIPIO"].apply(str)
df["ID_REGIAO"] = df["ID_REGIAO"].map(id_regiao)
df["ID_UF"] = df["ID_UF"].map(id_uf)
df["ID_AREA"] = df["ID_AREA"].map(id_area)
df["ID_LOCALIZACAO"] = df["ID_LOCALIZACAO"].map(id_localizacao)
df["NIVEL_SOCIO_ECONOMICO"] = df["NIVEL_SOCIO_ECONOMICO"].apply(lambda x : x.replace("í", "i").replace(" ", "-").lower())

In [74]:
binary_features_all = (
    [f"TX_RESP_Q0{i}" for i in range(34, 41)] +
    [f"TX_RESP_Q0{i}" for i in range(57, 67)] +
    [f"TX_RESP_Q0{i}" for i in range(76, 100)] +
    [f"TX_RESP_Q{i}" for i in range(100, 103)] +
    [f"TX_RESP_Q{i}" for i in range(109, 118)] +
    ["TX_RESP_Q129", "TX_RESP_Q131", "TX_RESP_Q133"] +
    [f"TX_RESP_Q{i}" for i in range(135, 147)] +
    [f"TX_RESP_Q{i}" for i in range(156, 159)] +
    [f"TX_RESP_Q{i}" for i in range(166, 199)] +
    [f"TX_RESP_Q{i}" for i in range(206, 252)]
)
binary_features = []


# should we make "inválido" a new class or make it equal to 0?
binary_invalid = {"A" : 1, "B" : 0, "invalido" : 0}
binary = {"A" : 1, "B": 0}
for col in binary_features_all:
  if not col in df.columns:
    continue
  binary_features.append(col)
  if df[col].isin(["invalido"]).max():
    df[col] = df[col].map(binary_invalid)
  else:
    df[col] = df[col].map(binary)

# fixing some categoric features
df["TX_RESP_Q067"] = df["TX_RESP_Q067"].apply(lambda x : 0 if x=="invalido" else x)
df["TX_RESP_Q067"] = df["TX_RESP_Q067"].astype("float")
for i in range(119, 125):
  df[f"TX_RESP_Q{i}"] = df[f"TX_RESP_Q{i}"].apply(lambda x : 0 if x=="invalido" else x)
  df[f"TX_RESP_Q{i}"] = df[f"TX_RESP_Q{i}"].astype("float")


In [75]:
target = "MEDIA"
num_features = df.columns[df.dtypes == "float64"]
cat_features = [col for col in df.columns if not (col in binary_features or col in num_features)]